In [1]:
using RCall # use R in Julia and thus in this jupyter notebook
using CSV # save/load txt files
using LowRankApprox # randomized low-rank approximation

INFO: Precompiling module RCall.
INFO: Using R installation at /Library/Frameworks/R.framework/Resources


In [2]:
L1 = Array{Float64,2}(CSV.read("../data/sample5000x20.txt", nullable = false, header = false, delim = ' '));

In [13]:
# L       : likelihood matrix; design matrix of size n by m
# x       : initial point with default (1/m, 1/m, ...)
# convtol :
# 
function mixSQP(L; x = ones(size(L,2))/size(L,2), convtol = 1e-8,
                pqrtol = 1e-8, eps = 1e-8, sptol = 1e-3,
                maxiter = 100, maxqpiter = 100,
                lowrank = "svd", seed = 1, verbose = true)
    
  # Get the number of rows (n) and columns (k) of L
  n = size(L,1); k = size(L,2);

  # If requested (i.e., if pqrtol > 0), compute the partial QR
  # decomposition with relative precision "tol", then retrieve the
  # permutation matrix, P. For details on the partial QR, see
  # https://github.com/klho/LowRankApprox.jl.
    
  # start timing for low-rank approximation of L
  tic();
  if lowrank == "qr"
      srand(seed)
      F = pqrfact(L, rtol=pqrtol);
      P = sparse(F[:P]);
  elseif lowrank == "svd"
      srand(seed)
      F = psvdfact(L, rtol=pqrtol);
      S = Diagonal(F[:S]);
  else
  end
  lowranktime = toq();
    
  # Summarize the analysis here.
  if verbose
    @printf("Running SQP algorithm with the following settings:\n")
    @printf("- %d x %d data matrix\n",n,k)
    @printf("- convergence tolerance = %0.2e\n",convtol)
    @printf("- zero threshold        = %0.2e\n",sptol)
    if lowrank == "qr"
      err = maximum(abs.(F[:Q]*F[:R]*P' - L));
      @printf("- partial QR tolerance  = %0.2e\n",pqrtol)
      @printf("- partial QR max. error = %0.2e\n",err)
    elseif lowrank == "svd"
      err = maximum(abs.(F[:U]*S*F[:Vt] - L));
      @printf("- partial SVD tolerance  = %0.2e\n",pqrtol)
      @printf("- partial SVD max. error = %0.2e\n",err)
    else
      @printf("- Exact derivative computation (partial QR not used).\n")
    end
  end

  # Initialize storage for the outputs obj, gmin, nnz and nqp.
  obj    = zeros(maxiter);
  gmin   = zeros(maxiter);
  nnz    = zeros(maxiter);
  nqp    = zeros(maxiter);
  timing = zeros(maxiter);
    
  # Initialize loop variables used in the loops below so that they
  # are available outside the scope of the loop.
  i = 0;
  j = 0;
  D = 0;

  # Print the column labels for reporting the algorithm's progress.
  if verbose
    @printf("iter       objective -min(g+1) #nnz #qp\n")
  end

  # QP subproblem start.
  for i = 1:maxiter

    # Start timing the iteration.
    tic();
      
    # Compute the gradient and Hessian, optionally using the partial
    # QR decomposition to increase the speed of these computations.
    # gradient and Hessian computation -- Rank reduction method
    if lowrank == "qr"
        D = 1./(F[:Q]*(F[:R]*(P'*x)) + eps);
        g = -P * F[:R]' * (F[:Q]'*D)/n;
        H = P * F[:R]' * (F[:Q]'*Diagonal(D.^2)*F[:Q]) * F[:R] * P'/n + eps * eye(k);
    elseif lowrank == "svd"
        D = 1./(F[:U]*(S*(F[:Vt]*x)) + eps);
        g = -F[:Vt]'*(S * (F[:U]'*D))/n;
        H = (F[:V]*S*(F[:U]'*Diagonal(D.^2)*F[:U])* S*F[:Vt])/n + eps * eye(k);
    else
        D = 1./(L*x + eps);
        g = -L'*D/n;
        H = L'*Diagonal(D.^2)*L/n + eps * eye(k);
    end

    # Report on the algorithm's progress.
    #
    # TO DO: The L * x matrix operation here used to compute the
    # objective function could dramatically slow down the algorithm
    # when number of QR factors in the partial QR is much smaller than
    # k. We need to think of a way to avoid this by having an option
    # to not output the objective function at each iteration, and/or
    # make sure that this objective function operation is not included
    # in the timing.
    #
    obj[i]  = -sum(log.(L * x + eps));
    gmin[i] = minimum(g + 1);
    nnz[i]  = sum(x .> sptol);
    nqp[i]  = j;
    if verbose
      @printf("%4d %0.8e %+0.2e %4d %3d\n",i,obj[i],-gmin[i],nnz[i],j);
    end
      
    # Check convergence of outer loop
    if minimum(g + 1) >= -convtol
      break
    end
      
    # Initialize the solution to the QP subproblem (y).
    ind    = find(x .> sptol);
    y      = sparse(zeros(k));
    y[ind] = 1/length(ind);

    # Run active set method to solve the QP subproblem.
    for j = 1:maxqpiter
          
      # Define the smaller QP subproblem.
      s   = length(ind);
      H_s = H[ind,ind];
      d   = H*y + 2*g + 1;
      d_s = d[ind];

      # Solve the smaller problem.
      p      = sparse(zeros(k));
      p_s    = -H_s\d_s;
      p[ind] = p_s;

      # Check convergence using KKT
      if norm(p_s) < convtol
            
        # Compute the Lagrange multiplier.
        lambda = d;
        if all(lambda .>= -convtol)
          break;
        else
            
          # TO DO: Explain what ind and ind_min are for.
          ind_min = findmin(lambda)[2];
          ind     = sort([ind; ind_min]);
        end
      else
          
        # Find a feasible step length.
        alpha     = 1;
        alpha0    = -y[ind]./p_s;
        ind_block = find(p_s .< 0);
        alpha0    = alpha0[ind_block];
        if ~isempty(ind_block)
          v, t = findmin(alpha0);
          if v < 1

            # Blocking constraint.
            ind_block = ind[ind_block[t]]; 
            alpha     = v;
              
            # Update working set if there is a blocking constraint.
            deleteat!(ind,find(ind - ind_block .== 0));
          end
        end
          
        # Move to the new "inner loop" iterate (y) along the search
        # direction.
        y = y + alpha * p;
      end
    end

    # Update the solution to the original optimization problem.
    x = y;

    # Get the elapsed time for the ith iteration.
    timing[i] = toq();
  end

  # Return: (1) the solution (after zeroing out any values below the
  # tolerance); (2) the value of the objective at each iteration; (3)
  # the minimum gradient value of the modified objective at each
  # iteration; (4) the number of nonzero entries in the vector at each
  # iteration; and (5) the number of inner iterations taken to solve
  # the QP subproblem at each outer iteration.
  x[x .< sptol] = 0; x = x/sum(x);
  totaltime = lowranktime + sum(timing[1:i]);
  if verbose
    @printf("Optimization took %d iterations and %0.4f seconds.\n",i,totaltime)
  end

  return Dict([("x",full(x)), ("totaltime",totaltime), ("lowranktime",lowranktime),
               ("obj",obj[1:i]), ("gmin",gmin[1:i]), ("nnz",nnz[1:i]),
               ("nqp",nqp[1:i]), ("timing",timing[1:i])])
end

mixSQP (generic function with 1 method)

In [14]:
mixSQP(L1)

Running SQP algorithm with the following settings:
- 5000 x 20 data matrix
- convergence tolerance = 1.00e-08
- zero threshold        = 1.00e-03
- partial SVD tolerance  = 1.00e-08
- partial SVD max. error = 5.39e-08
iter       objective -min(g+1) #nnz #qp
   1 2.95206790e+03 +1.81e-01   20   0
   2 2.55805274e+03 +2.20e-01    3  22
   3 2.37809188e+03 +4.08e-02    3   2
   4 2.34432187e+03 +5.72e-03    3   2
   5 2.33599066e+03 +2.59e-04    3   2
   6 2.33556636e+03 +4.81e-07    3   2
   7 2.33556553e+03 -9.52e-09    3   2
Optimization took 7 iterations and 0.0990 seconds.


Dict{String,Any} with 8 entries:
  "timing"      => [0.0854581, 0.000888117, 0.000801054, 0.00116372, 0.00114653…
  "nnz"         => [20.0, 3.0, 3.0, 3.0, 3.0, 3.0, 3.0]
  "totaltime"   => 0.0990484
  "lowranktime" => 0.0084602
  "obj"         => [2952.07, 2558.05, 2378.09, 2344.32, 2335.99, 2335.57, 2335.…
  "x"           => [0.469905, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0,…
  "nqp"         => [0.0, 22.0, 2.0, 2.0, 2.0, 2.0, 2.0]
  "gmin"        => [-0.181487, -0.220214, -0.0407744, -0.00572226, -0.000259016…

In [5]:
include("../code/julia/mixSQP_time.jl")
include("../code/julia/mixEM.jl");
include("../code/julia/REBayes.jl");
include("../code/julia/makedata.jl");

In [20]:
srand(1)
temp = pqrfact(L1, rank = 5);
L = temp[:Q] * temp[:R] * temp[:P]';

In [21]:
[rank(L1) rank(L)]

1×2 Array{Int64,2}:
 17  5

In [22]:
# time comparison between mixEM, mixSQP and REBayes

## mixEM;
tic(); x_em = mixEM(L)[1]; t_em = toq();

x_rebayes,t_rebayes = REBayes(L);

tic(); x_mixsqp = mixSQP_time(L)[1]; t_mixsqp = toq();

# mixSQP ourperforms on this small dataset 5000x20
["mixEM" "REbayes" "mixSQP"; t_em t_rebayes t_mixsqp]

2×3 Array{Any,2}:
   "mixEM"   "REbayes"   "mixSQP"
 17.3617    0.081       0.0420426

In [23]:
# EM does not converge until maxiter
[x_em x_rebayes x_mixsqp]

20×3 Array{Float64,2}:
 0.358333      0.35897    0.447119 
 0.00397846    0.0        0.0      
 0.000109288   0.0        0.0      
 1.99013e-7    0.0        0.0      
 4.40543e-12   0.0        0.0      
 1.63095e-19   0.0        0.0      
 5.72183e-30   0.0        0.0      
 6.50616e-41   0.0        0.0      
 7.34051e-44   0.0        0.0      
 8.55295e-28   0.0        0.0      
 0.251122      0.267272   0.0369685
 0.0592966     0.0433985  0.111889 
 2.28634e-24   0.0        0.0755728
 0.281933      0.286772   0.32845  
 0.0452267     0.0435877  0.0      
 1.39568e-228  0.0        0.0      
 1.97626e-323  0.0        0.0      
 4.94066e-324  0.0        0.0      
 0.0           0.0        0.0      
 0.0           0.0        0.0      

In [25]:
minf = min.(eval_f(L1,x_em), eval_f(L1,x_rebayes),  eval_f(L1,x_mixsqp));
[eval_f(L1,x_em), eval_f(L1,x_rebayes),  eval_f(L1,x_mixsqp)]

3-element Array{Float64,1}:
 0.470621
 0.470621
 0.467781